# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty via cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [ ]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [ ]:
def polynomial_sframe(feature, degree):
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living first:

In [ ]:
sales = sales.sort('sqft_living')

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. After fitting the model, print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=0.0` in this call.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use the `.random_split` function and make sure you set `seed=1`. 

In [ ]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=1)
(set_1, set_2) = semi_split1.random_split(0.5, seed=1)
(set_3, set_4) = semi_split2.random_split(0.5, seed=1)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: To make sure GraphLab Create doesn't introduce any regularization, make sure you include the option `l2_penalty=0.0` when calling `graphlab.linear_regression.create()`.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.) In GraphLab Create, adding an L2 penalty is a matter of adding an extra argument.

With the extra argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the extra parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

These curves should vary a lot less, now that you introduced regularization.

***QUIZ QUESTION:  For the models learned with regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. To address this, we will go back to using a validation set. 

What we are going to do here is:

* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [ ]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next you should write a loop that does the following:
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Fit a polynomial regression model to sqft vs price with that `l2_penalty` on TRAIN data. (Specificy `l2_penalty=l2_penalty` in the parameter list.)
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l2_penalty`
* Report which L2 penalty produced the lowest RSS on validation data (remember python indexes from 0)

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`.  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop.

In [ ]:
import numpy as np

Tip: If you are plotting L2 penalty vs RSS, we recommend that you call  `plt.xscale('log')` to draw the x-axis in log scale.

Now that you have chosen the L2 penalty using validation data, compute the RSS of this model on TEST data. 

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to the validation data?  What is the RSS on the TEST data of the model you learn with this L2 penalty? ***